In [18]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import numpy as np
import math

In [ ]:
sample_list=['sample_tmp']
input_dir = '/data/work/TLS_shells'

In [ ]:
#Calculate score data

In [ ]:
for sample in sample_list:
    score_file = pd.read_csv(f'{input_dir}/score_data/{sample}_obs_Score.csv')
    score_columns = [col for col in score_file.columns if col.endswith('_score')]
    score_columns.append('cellbin_pos')
    score_data = score_file[score_columns]
    
    expansion_file = pd.read_csv(f'{input_dir}/result/{sample}/{sample}.expansion.only1200um.csv',index_col=0)
    TLS_namelist = expansion_file['TLS_name'].unique().astype('str').tolist()
    TLSnames = [x for x in TLS_namelist if 'TLS' in x]
    for TLS in TLSnames:
        expansion_col = f'{TLS}_expansion'
        expansion_data = expansion_file[(expansion_file[expansion_col]!='other')&(expansion_file['TLS_name'].isin([TLS,'other']))]
        expansion_score = pd.merge(expansion_data, score_data, on = ['cellbin_pos'], how = 'left')
        score_columns.append(expansion_col)
        score_columns.append('cell2loc_anno')
        expansion_score = expansion_score[score_columns].rename(columns={expansion_col: 'expansion_size'})
        expansion_score['expansion_size'] = expansion_score['expansion_size'].astype(float) 
        expansion_score = expansion_score[expansion_score['expansion_size']<=2000.0]
        expansion_score['TLS_name'] = TLS
        expansion_score.to_csv(f'{input_dir}/fitcurve/fit_data/score/{expansion_col}.only1200um.csv',index=False)

In [ ]:
score_list = ['Mac_FOLR2_score','Mac_GPNMB_score','TLS_name','expansion_size']

all_result=[]
fitcurveDir = glob.glob(f'{input_dir}/fitcurve/fit_data/score/*.only1200um.csv')
for scorePath in fitcurveDir:
    score_file = pd.read_csv(scorePath)
    score_data = score_file[score_list]
    all_result.append(score_data)
all_data = pd.concat(all_result)
mean_data = all_data.groupby(['TLS_name','expansion_size']).mean().reset_index()
mean_data.to_csv(f'{input_dir}/fitcurve/plot/expansion.only1200um_score.csv',index=False)

/tmp/ipykernel_597/3799596080.py:10: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  score_file = pd.read_csv(scorePath)
/tmp/ipykernel_597/3799596080.py:10: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  score_file = pd.read_csv(scorePath)
/tmp/ipykernel_597/3799596080.py:10: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  score_file = pd.read_csv(scorePath)
/tmp/ipykernel_597/3799596080.py:10: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  score_file = pd.read_csv(scorePath)
/tmp/ipykernel_597/3799596080.py:10: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  score_file = pd.read_csv(scorePath)


In [ ]:
#cell proportions
#The input file contains cell count data within circular regions of radii 100μm, 200μm, and 300μm, etc. with column headers such as 'TLS_name B_cell T_cell'.

In [ ]:
input_path = f'{input_dir}/singleCelltype/stat'
output_path = f'{input_dir}/fitcurve/fit_data/cell_rate'
os.makedirs(output_path,exist_ok=True)

In [ ]:
size_list=[100,200,300,400,500,600,700,800,900,1000,1100,1200]
first_size = size_list[0]
first_df = pd.read_csv(f'{input_path}/TLSexpansion_cellnumber.{first_size}um.csv')
first_df['size'] = 100
first_df.to_csv(f'{output_path}/TLSexpansion_cellnumber.ring_0to{first_size}um.csv', index=False)
for i in range(1, len(size_list)):
    prev_size = size_list[i-1]
    current_size = size_list[i]

    current_df = pd.read_csv(f'{input_path}/TLSexpansion_cellnumber.{current_size}um.csv')
    prev_df = pd.read_csv(f'{input_path}/TLSexpansion_cellnumber.{prev_size}um.csv')

    ring_df = current_df.copy()
    for col in ring_df.columns:
        if col != 'orig.ident' and col in prev_df.columns:
            ring_df[col] = ring_df[col] - prev_df[col]
    ring_df['size'] = current_size
    ring_df.to_csv(f'{output_path}/TLSexpansion_cellnumber.ring_{prev_size}to{current_size}um.csv', index=False)

In [ ]:
allresult=[]
ring_paths = glob.glob(f'{output_path}/*um.csv')
for ring_path in ring_paths:
    ring_file = pd.read_csv(ring_path)
    ring_name = os.path.basename(ring_path).replace('.csv','')
    celltype_cols = [x for x in ring_file.columns if x not in ['orig.ident', 'size']]
    ring_file['allcount'] = ring_file[celltype_cols].sum(axis=1)
    for col in celltype_cols:
        new_col_name = f"{col}_rate"
        ring_file[new_col_name] = ring_file[col] / ring_file['allcount']
        ring_file = ring_file.fillna(0)
    ring_file.to_csv(f'{output_path}/{ring_name}.rate.csv',index=False)
    allresult.append(ring_file)
allresult_file = pd.concat(allresult)
allresult_file['Mature'] = allresult_file['orig.ident'].map(mature_dict)
allresult_file.to_csv(f'{output_path}/allTLSexpansion_CellRate.csv',index=False)